In [2]:
import requests

page = requests.get('https://gg.deals/game/tricky-towers/')
game_html=page.text

In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(game_html, 'lxml')

In [12]:
soup.find_all('span', itemprop='name')

[<span itemprop="name">Home</span>,
 <span itemprop="name">Games</span>,
 <span itemprop="name">Casual</span>,
 <span itemprop="name">Tricky Towers</span>]

In [13]:
soup.find('img',class_="image-game")['src']

'https://img.gg.deals/70/ae/26c780669e69d3cd65f6bcd9f5aefa6dfaa6_307xt176.jpg'

In [15]:
soup.find_all('p',class_="game-info-details-content")

[<p class="game-info-details-content">02 Aug 2016</p>,
 <p class="game-info-details-content">WeirdBeard</p>]

In [16]:
soup.find_all('span',class_="overlay")

[<span class="overlay">80</span>, <span class="overlay">6.8</span>]

In [19]:
soup.find_all('span',class_="reviews-label")

[<span class="reviews-label" data-tippy-animatefill="false" data-tippy-arrow="false" data-tippy-duration="0" data-tippy-followcursor="1" data-tippy-theme="translucent" title="90% of the 6,630 user reviews for this game are positive.">Very Positive<span class="semi-transparent"> (6,630)</span></span>]

In [21]:
soup.find_all('a',class_="badge")

[<a class="badge red-badge hoverable white-text" data-class="risk-modal" data-dismiss="modal" data-target="#ajax-modal" data-toggle="modal" data-url="/ru/shops/risks/63/" href="#">8 risks</a>,
 <a class="badge red-badge hoverable white-text" data-class="risk-modal" data-dismiss="modal" data-target="#ajax-modal" data-toggle="modal" data-url="/ru/shops/risks/62/" href="#">8 risks</a>,
 <a class="badge red-badge hoverable white-text" data-class="risk-modal" data-dismiss="modal" data-target="#ajax-modal" data-toggle="modal" data-url="/ru/shops/risks/60/" href="#">3 risks</a>,
 <a class="badge red-badge hoverable white-text" data-class="risk-modal" data-dismiss="modal" data-target="#ajax-modal" data-toggle="modal" data-url="/ru/shops/risks/59/" href="#">1 risk</a>,
 <a class="badge" href="/games/?genre=10">Casual</a>,
 <a class="badge" href="/games/?genre=1">Indie</a>,
 <a class="badge" href="/games/?tag=422">Party Game</a>,
 <a class="badge" href="/games/?tag=10">Casual</a>,
 <a class="bad

In [22]:
soup.find_all('section',id_="game_dlcs")

[]

In [23]:
soup.find_all('a',id_="requirement-tab-trigger-pc-requirements")

[]

In [5]:
def process_page(url):
    soup=BeautifulSoup(requests.get(url).text, 'lxml')
    my_dict={
            'url':url,
            'name':soup.find_all('span', itemprop='name')[3].text,
            'img':soup.find('img',class_="image-game")['src']
    }
    return my_dict

In [6]:
process_page('https://gg.deals/game/resident-evil-5/')

{'url': 'https://gg.deals/game/resident-evil-5/',
 'name': 'Resident Evil 5',
 'img': 'https://img.gg.deals/34/15/1f0b040d7435357286447aaff259898cf2ae_307xt176.jpg'}

In [7]:
mas_of_kartochki=[]
mas_of_urls=[]
soup = BeautifulSoup(requests.get('https://gg.deals/games/?type=1&page=1').text, 'lxml')
mas_of_candidats=soup.find_all('a', class_="main-image")
for cand in mas_of_candidats:
    tek_url='https://gg.deals' + str(cand['href'])
    mas_of_urls.append(tek_url)
    mas_of_kartochki.append(process_page(tek_url))
    
mas_of_kartochki

[{'url': 'https://gg.deals/game/days-gone/',
  'name': 'Days Gone',
  'img': 'https://img.gg.deals/38/22/2e05636aadb3da72d2a06eb902ac1667dbbd_307xt176.jpg'},
 {'url': 'https://gg.deals/game/tekken-7/',
  'name': 'TEKKEN 7',
  'img': 'https://img.gg.deals/db/7f/9c9c5a85b6a0be5cb9b8353c643fa1c19b6b_307xt176.jpg'},
 {'url': 'https://gg.deals/game/outriders/',
  'name': 'OUTRIDERS',
  'img': 'https://img.gg.deals/b8/85/ce85db8fc7ec96b3e1bac6eeadd71fc466a3_307xt176.jpg'},
 {'url': 'https://gg.deals/game/the-forest/',
  'name': 'The Forest',
  'img': 'https://img.gg.deals/8c/77/1f40768dcebbad3bde5b562193648b848e0c_307xt176.jpg'},
 {'url': 'https://gg.deals/game/filament/',
  'name': 'Filament',
  'img': 'https://img.gg.deals/18/55/adac90bb63ab0f71c88d65596f6afbd42ea6_307xt176.jpg'},
 {'url': 'https://gg.deals/game/horizon-zero-dawn-complete-edition/',
  'name': 'Horizon Zero Dawn Complete Edition',
  'img': 'https://img.gg.deals/0a/46/221cb09ac5debb227c08fc2303fbebd6c3f9_307xt176.jpg'},
 {'u

In [12]:
import gzip
import json
import codecs
from tqdm import tqdm
from multiprocessing.dummy import Pool, Queue

queue = Queue()
for el in mas_of_urls:
    queue.put(el)  # очередь ссылок на книги


def process_page_wrapper(nomer_potoka):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(nomer_potoka), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            record = process_page(queue.get())
            print(record)
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=2) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

  8%|▊         | 2/24 [00:00<00:08,  2.47it/s]

{'url': 'https://gg.deals/game/days-gone/', 'name': 'Days Gone', 'img': 'https://img.gg.deals/38/22/2e05636aadb3da72d2a06eb902ac1667dbbd_307xt176.jpg'}
{'url': 'https://gg.deals/game/tekken-7/', 'name': 'TEKKEN 7', 'img': 'https://img.gg.deals/db/7f/9c9c5a85b6a0be5cb9b8353c643fa1c19b6b_307xt176.jpg'}


 12%|█▎        | 3/24 [00:00<00:07,  2.70it/s]

{'url': 'https://gg.deals/game/outriders/', 'name': 'OUTRIDERS', 'img': 'https://img.gg.deals/b8/85/ce85db8fc7ec96b3e1bac6eeadd71fc466a3_307xt176.jpg'}


 17%|█▋        | 4/24 [00:01<00:07,  2.73it/s]

...

{'url': 'https://gg.deals/game/satisfactory/', 'name': 'Satisfactory', 'img': 'https://img.gg.deals/39/27/d08d787969917edda0e88ad45a3790030126_307xt176.jpg'}


 96%|█████████▌| 23/24 [00:07<00:00,  3.22it/s]

{'url': 'https://gg.deals/game/resident-evil-village/', 'name': 'Resident Evil Village', 'img': 'https://img.gg.deals/52/22/084c4619ec25c30c42365ab682b6293bb4e6_307xt176.jpg'}


100%|██████████| 24/24 [00:07<00:00,  3.51it/s]

{'url': 'https://gg.deals/game/ghostrunner/', 'name': 'Ghostrunner', 'img': 'https://img.gg.deals/12/17/05a7be4d6af388cf06390859168958697eb8_307xt176.jpg'}


In [3]:
href="https://gg.deals/redirect/2a8957805c99632e75cf1a2b18826629773e0355/?utm_source=games%2Fsingle"
response = requests.get(href)

In [4]:
response.url

'https://store.steampowered.com/app/437920/'